In [4]:
# Import the required libraries to load the dataset
import os
import random
import shutil
import uuid

# Define the destination directories
train_directory = 'train'
test_directory = 'test'

# Define the paths to the source directories
classes = ['dogs', 'food', 'vehicles']

# Create the train and test directories if they don't already exist
for directory in [train_directory, test_directory]:
    os.makedirs(os.path.join(directory, classes[0]), exist_ok=True)
    os.makedirs(os.path.join(directory, classes[1]), exist_ok=True)
    os.makedirs(os.path.join(directory, classes[2]), exist_ok=True)


In [5]:
# Define the training and testing split ratios

train_split = 0.8
test_split = 0.2

In [6]:
# Loop through each source directory
for data_dir in classes:

    # Get the path to the current source directory
    path = os.path.join('cnn_dataset', data_dir)
    
    # Get a list of all the image files in the current source directory
    image_files = []
    for image in os.listdir(path):
      image_files.append(os.path.join(path, image))
    
    # Randomly shuffle the image files
    random.shuffle(image_files)
    
    # Calculate the number of images to include in the training set
    train_images_size = int(train_split * len(image_files))
    test_images_size = int(test_split * len(image_files))
    
    # Split the image files into training and testing sets
    train_images = image_files[:train_images_size]
    test_images = image_files[:test_images_size]
    
    # Move the training images to the train directory
    for img in train_images:
        filename = str(uuid.uuid4()) + '_' + os.path.basename(img)
        final_path = os.path.join(train_directory, data_dir, filename)
        shutil.copy(img, final_path)
        
    # Move the testing images to the test directory
    for img in test_images:
        filename = str(uuid.uuid4()) + '_' + os.path.basename(img)
        final_path = os.path.join(test_directory, data_dir, filename)
        shutil.copy(img, final_path)

In [7]:
import torch
from torchvision.transforms import transforms

# Define the data transforms for the training set
train_augmentations = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(size = 224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define the data transforms for the validation set
test_transformations = transforms.Compose([
    transforms.Resize(size = 224),
    transforms.CenterCrop(size = 224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [8]:
from torchvision.datasets import ImageFolder

# Define the datasets for the train and validation sets
train_dataset = ImageFolder(root = 'train', transform = train_augmentations)
test_dataset = ImageFolder(root = 'test', transform = test_transformations)

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class MyVGG(nn.Module):
    def __init__(self):
      super().__init__()
        
      self.features = nn.Sequential(
          
        nn.Conv2d(3, 64, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        
        nn.Conv2d(64, 128, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        
        nn.Conv2d(128, 256, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        
        nn.Conv2d(256, 512, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        
        nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
      )
        
      self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        
      self.classifier = nn.Sequential(
        nn.Linear(512 * 7 * 7, 4096),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(4096, 3),
      )

    def forward(self, x):
      x = self.features(x)
      x = self.avgpool(x)
      x = x.view(x.size(0), -1)
      x = self.classifier(x)
      return x


In [10]:
from torch.utils.data import DataLoader

# Define the data loaders for the train and validation sets
train_loader = DataLoader(train_dataset, batch_size = 128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size = 128)

In [11]:
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model and move it to the device
model = MyVGG().to(device)

In [12]:
from tqdm.auto import tqdm

def train_model(model, epochs, train_loader, test_loader):

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  # Initializing training loss, training accuracy list
  train_loss_vals = []
  train_accuracy_vals = []

  # Initializing testing loss, testing accuracy list
  test_loss_vals = []
  test_accuracy_vals = []

  learning_rate = 1e-5
  loss_fun = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

  # Define the loss function and optimizer
  criterion = loss_fun
  optimizer = optimizer

  # Train the model for the specified number of epochs
  for epoch in range(epochs):

      # Set the model to training mode
      model.train()

      # Initializing actual values to zero to compute training accuracy
      actual_train_val = 0
      actual_test_val = 0

      # Initializing training loss for computations
      train_loss = 0

      # Initializing testing loss for computations
      test_loss = 0

      # Loop over the training set in batches
      for pred_vals, true_vals in tqdm(train_loader):

        # Convert these pred_vals, true_vals to device to be compatible
        pred_vals = pred_vals.to(device)
        true_vals = true_vals.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Compute the forward pass through the neural network
        result = model(pred_vals)

        # Compute the loss value
        loss_vals = criterion(result, true_vals)

        # Compute the gradients using backpropagation
        loss_vals.backward()

        # Update the weights of the network using the step method of optimizer
        optimizer.step()

        # Add the batch loss to the training loss
        train_loss += loss_vals.item()*pred_vals.size(0)

        # Computing the training accuracy
        max_val = torch.argmax(result, dim = 1)
        actual_train_val += torch.eq(max_val, true_vals).sum().item()

      train_loss = train_loss/len(train_loader.dataset)
      actual_train_val = actual_train_val/len(train_loader.dataset)

      train_loss_vals.append(train_loss)
      train_accuracy_vals.append(actual_train_val)

      # Set the model to evaluation model
      model.eval()

      with torch.no_grad():

        for pred_vals, true_vals in test_loader:
            
            # Move prediction values and true values to the same device as model
            pred_vals = pred_vals.to(device)
            true_vals = true_vals.to(device)

            result = model(pred_vals)

            # Computing the training accuracy
            max_val = torch.argmax(result, dim = 1)
            actual_test_val += torch.eq(max_val, true_vals).sum().item()

      test_loss = test_loss/len(test_loader.dataset)
      actual_test_val = actual_test_val/len(test_loader.dataset)

      test_loss_vals.append(test_loss)
      test_accuracy_vals.append(actual_test_val)

      # Printing Stats for every epoch
      print('Training loss, Training accuracy, and Testing accuracy for epoch {} are {:.3f}, {:.3f} and, {:.3f}'.format(epoch + 1, train_loss, actual_train_val*100, actual_test_val*100))

  return model, train_loss_vals, test_loss_vals, train_accuracy_vals, test_accuracy_vals

In [14]:
epochs = 10

# Executing the VGG13 model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MyVGG().to(device)

# Using CrossEntropy Loss and Adam optimizer for best accuracy
model, train_loss_vals, test_loss_vals, train_accuracy_vals, test_accuracy_vals = train_model(model, epochs, train_loader, test_loader)

  0%|          | 0/188 [00:00<?, ?it/s]

Training loss, Training accuracy, and Testing accuracy for epoch 1 are 0.970, 48.083 and, 64.167


  0%|          | 0/188 [00:00<?, ?it/s]

Training loss, Training accuracy, and Testing accuracy for epoch 2 are 0.751, 66.163 and, 71.617


  0%|          | 0/188 [00:00<?, ?it/s]

...

Training loss, Training accuracy, and Testing accuracy for epoch 8 are 0.520, 79.208 and, 79.767


  0%|          | 0/188 [00:00<?, ?it/s]

Training loss, Training accuracy, and Testing accuracy for epoch 9 are 0.503, 80.350 and, 84.483


  0%|          | 0/188 [00:00<?, ?it/s]

Training loss, Training accuracy, and Testing accuracy for epoch 10 are 0.486, 80.979 and, 82.233


In [18]:
def evaluate_performance(model, test_dataloader): 
  
  accuracy = 0

  # Set the model to evaluation model
  model.eval()

  with torch.no_grad():

    for pred_vals, true_vals in test_dataloader:
        
        # Move prediction values and true values to the same device as model
        pred_vals = pred_vals.to(device)
        true_vals = true_vals.to(device)

        result = model(pred_vals)

        # Computing the training accuracy
        max_val = torch.argmax(result, dim = 1)
        accuracy += torch.eq(max_val, true_vals).sum().item()

  accuracy = accuracy/len(test_loader.dataset)

  return accuracy

In [19]:
# test accuracy

test_accuracy_vgg = evaluate_performance(model, test_loader)

print("Test accuracy for model using Adam optimizer for VGG model is {:.3f}".format(test_accuracy_vgg*100))


Test accuracy for model using Adam optimizer for VGG model is 82.233
